### connect llm

In [1]:
from langchain_community.llms import Ollama
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

models = ["deepseek-coder-v2", "llama3", "gemma2", "deepseek-v2"]
model = models[0]

/Volumes/t7/mac/gitProjects/bank-tx-classification/llm_venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
llm = Ollama(
    model=model
)

### load bank txs and combine into one file

In [3]:
import pandas as pd
import glob

csv_names = glob.glob('data/*.csv')
dfs = []
header_names = ["Date", "Amount", "Desc"]

for fn in csv_names:
    df = pd.read_csv(fn, names=header_names, header=None)
    dfs.append(df)

combined_df = pd.concat(dfs)
combined_df = combined_df[0:20]
combined_df

,Date,Amount,Desc
0,27/09/2017,-40.85,HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Va...
1,27/09/2017,-6.50,DELISSE SYDNEY NS AUS Card xx7113 Value Date: ...
2,27/09/2017,-4.00,SOUTHGATE GROUP AUSTRA SYDNEY AUS Card xx7113...
3,27/09/2017,-10.00,Deli Ziosa Sydney NS AUS Card xx7113 Value Dat...
4,27/09/2017,-0.80,DELISSE SYDNEY NS AUS Card xx7113 Value Date: ...
5,27/09/2017,-430.00,BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7...
6,27/09/2017,-4.00,LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 V...
7,26/09/2017,-59.99,Direct Debit 142619 TPG Internet DH9Q0HB22EP-0...
8,26/09/2017,-17.85,LUO & SONG CHICKEN QPS PEAKHURST AUS Card xx8...
9,26/09/2017,-34.76,PLINE PH SUTHERLAND SUTHERLAND AUS Card xx873...


### predefined categories

In [4]:
predefined_categories = [
    "Groceries",
    "Transportation",
    "Dining Out",
    "Coffee & Snacks",
    "Personal Care",
    "Entertainment",
    "Health & Fitness",
    "Household Supplies",
    "Clothing",
    "Childcare",
    "Pet Care",
    "Miscellaneous"
]

### embeddings of predefined categories

In [5]:
import requests

def embed(prompt:str) -> list:
    url = "http://localhost:11434/api/embeddings"
    data = {
        "model": model,
        "prompt": prompt
    }
    response = requests.post(url, json=data)
    return response.json()

embed("text to embed")

{'embedding': [-0.14101070165634155,
  -0.2909574508666992,
  0.13802006840705872,
  0.4165777266025543,
  0.10250671952962875,
  -0.07069890201091766,
  0.41344666481018066,
  0.29414331912994385,
  0.16599515080451965,
  0.042373523116111755,
  0.3521226942539215,
  0.03833143785595894,
  -0.13629257678985596,
  -0.0872301310300827,
  0.045080143958330154,
  -0.35673946142196655,
  -0.035183507949113846,
  -0.04551728442311287,
  -0.2036796361207962,
  0.14169764518737793,
  0.14461684226989746,
  -0.31405264139175415,
  -0.17079240083694458,
  -0.2268713414669037,
  -0.07935179024934769,
  0.1671672910451889,
  0.05626717582345009,
  -0.28962287306785583,
  -0.21314489841461182,
  0.1151994988322258,
  0.10502927750349045,
  0.120858334004879,
  -0.02105855755507946,
  0.01821432076394558,
  -0.012033175677061081,
  -0.19742722809314728,
  -0.009497363120317459,
  -0.2753775417804718,
  0.01811690628528595,
  -0.1643543392419815,
  0.09502504020929337,
  -0.15839874744415283,
  0.06

In [6]:
embeddings = [];
for c in predefined_categories:
    embeddings.append(embed(c)["embedding"])

embeddings

[[0.2256433516740799,
  -0.526540994644165,
  -0.17445018887519836,
  0.13835349678993225,
  0.1768334060907364,
  0.40465569496154785,
  0.015767183154821396,
  0.037455543875694275,
  0.18559682369232178,
  0.11994358897209167,
  -0.13714599609375,
  0.02263568714261055,
  -0.008258416317403316,
  -0.3272227942943573,
  -0.06201435998082161,
  -0.24452926218509674,
  -0.05047544836997986,
  0.15533825755119324,
  0.33868587017059326,
  -0.28001487255096436,
  0.2574114501476288,
  -0.12688076496124268,
  -0.11092693358659744,
  -0.17584478855133057,
  -0.19828693568706512,
  0.05338648334145546,
  0.573833167552948,
  -0.0732833594083786,
  -0.17881202697753906,
  -0.08030819892883301,
  0.22235682606697083,
  -0.025418300181627274,
  0.04520626366138458,
  -0.08444896340370178,
  0.009756308048963547,
  -0.2750803530216217,
  -0.1714840978384018,
  0.01815769262611866,
  -0.28814005851745605,
  -0.18374182283878326,
  0.020048994570970535,
  0.1270519196987152,
  0.10655821114778519

In [7]:
df = pd.DataFrame({"category": predefined_categories, "embedding": embeddings})

### by cos sim

In [8]:
te = embed("HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017")
te["embedding"]

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

vec1 = np.array(df["embedding"].tolist())
vec2 = np.array([te["embedding"]])

cos_sim = cosine_similarity(vec1, vec2)
index_of_largest = np.argmax(cos_sim.flatten())

predefined_categories[index_of_largest]
### TBC use an different embedding model and try

'Coffee & Snacks'

In [9]:
def extract_between_strings(text, start, end):
    start_idx = text.find(start)
    if start_idx == -1:
        return None
    start_idx += len(start)
    
    end_idx = text.find(end, start_idx)
    if end_idx == -1:
        return None
    
    return text[start_idx:end_idx]

In [10]:
unique_descs = combined_df["Desc"].unique()
display(unique_descs)
len(unique_descs)

array(['HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017',
       'DELISSE SYDNEY NS AUS Card xx7113 Value Date: 25/09/2017',
       'SOUTHGATE GROUP AUSTRA SYDNEY  AUS Card xx7113 Value Date: 26/09/2017',
       'Deli Ziosa Sydney NS AUS Card xx7113 Value Date: 25/09/2017',
       'BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017',
       'LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 Value Date: 25/09/2017',
       'Direct Debit 142619 TPG Internet DH9Q0HB22EP-03GJKN',
       'LUO & SONG CHICKEN QPS PEAKHURST  AUS Card xx8731 Value Date: 23/09/2017',
       'PLINE PH SUTHERLAND SUTHERLAND  AUS Card xx8731 Value Date: 20/09/2017'],
      dtype=object)

9

### by prompt

In [11]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.messages import HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
import json

parser = StrOutputParser()
classified_desc = []
tx_desc = unique_descs.tolist()

for i in range(0, len(tx_desc), 5):
    print("======", i)
    tx_desc_group = tx_desc[i:i+5]
    
    template = f"""You're an expert of classifying a piece of bank transaction description.
    I will send you a list of bank descriptions in an array. 
    The classification result must be from this list {predefined_categories}
    This is the given bank description {tx_desc_group}.
    
    Please return a json in the following format: The json takes two attributes: one is called 'desc', 
    which is to store the original bank description, 
    another one is called classifcation, which is the result.
    
    Please return only one json array per response in json markdown format.
    """
    display(template)
    chain = llm | parser
    res = chain.invoke([HumanMessage(content=template)])
    display(res)
    classified_desc.extend(json.loads(extract_between_strings(res, "```json\n", "\n```")))

====== 0


"You're an expert of classifying a piece of bank transaction description.\n    I will send you a list of bank descriptions in an array. \n    The classification result must be from this list ['Groceries', 'Transportation', 'Dining Out', 'Coffee & Snacks', 'Personal Care', 'Entertainment', 'Health & Fitness', 'Household Supplies', 'Clothing', 'Childcare', 'Pet Care', 'Miscellaneous']\n    This is the given bank description ['HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017', 'DELISSE SYDNEY NS AUS Card xx7113 Value Date: 25/09/2017', 'SOUTHGATE GROUP AUSTRA SYDNEY  AUS Card xx7113 Value Date: 26/09/2017', 'Deli Ziosa Sydney NS AUS Card xx7113 Value Date: 25/09/2017', 'BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017'].\n    \n    Please return a json in the following format: The json takes two attributes: one is called 'desc', \n    which is to store the original bank description, \n    another one is called classifcation, which is the result.\n

' ```json\n[\n  {\n    "desc": "HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017",\n    "classification": "Groceries"\n  },\n  {\n    "desc": "DELISSE SYDNEY NS AUS Card xx7113 Value Date: 25/09/2017",\n    "classification": "Coffee & Snacks"\n  },\n  {\n    "desc": "SOUTHGATE GROUP AUSTRA SYDNEY  AUS Card xx7113 Value Date: 26/09/2017",\n    "classification": "Miscellaneous"\n  },\n  {\n    "desc": "Deli Ziosa Sydney NS AUS Card xx7113 Value Date: 25/09/2017",\n    "classification": "Groceries"\n  },\n  {\n    "desc": "BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017",\n    "classification": "Groceries"\n  }\n]\n```'

====== 5


"You're an expert of classifying a piece of bank transaction description.\n    I will send you a list of bank descriptions in an array. \n    The classification result must be from this list ['Groceries', 'Transportation', 'Dining Out', 'Coffee & Snacks', 'Personal Care', 'Entertainment', 'Health & Fitness', 'Household Supplies', 'Clothing', 'Childcare', 'Pet Care', 'Miscellaneous']\n    This is the given bank description ['LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 Value Date: 25/09/2017', 'Direct Debit 142619 TPG Internet DH9Q0HB22EP-03GJKN', 'LUO & SONG CHICKEN QPS PEAKHURST  AUS Card xx8731 Value Date: 23/09/2017', 'PLINE PH SUTHERLAND SUTHERLAND  AUS Card xx8731 Value Date: 20/09/2017'].\n    \n    Please return a json in the following format: The json takes two attributes: one is called 'desc', \n    which is to store the original bank description, \n    another one is called classifcation, which is the result.\n    \n    Please return only one json array per response in json m

' ```json\n[\n    {\n        "desc": "LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 Value Date: 25/09/2017",\n        "classification": "Dining Out"\n    },\n    {\n        "desc": "Direct Debit 142619 TPG Internet DH9Q0HB22EP-03GJKN",\n        "classification": "Miscellaneous"\n    },\n    {\n        "desc": "LUO & SONG CHICKEN QPS PEAKHURST  AUS Card xx8731 Value Date: 23/09/2017",\n        "classification": "Dining Out"\n    },\n    {\n        "desc": "PLINE PH SUTHERLAND SUTHERLAND  AUS Card xx8731 Value Date: 20/09/2017",\n        "classification": "Dining Out"\n    }\n]\n```'

In [58]:
print(classified_desc)
len(classified_desc)

[{'desc': 'HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017', 'classification': 'Groceries'}, {'desc': 'DELISSE SYDNEY NS AUS Card xx7113 Value Date: 25/09/2017', 'classification': 'Coffee & Snacks'}, {'desc': 'SOUTHGATE GROUP AUSTRA SYDNEY  AUS Card xx7113 Value Date: 26/09/2017', 'classification': 'Miscellaneous'}, {'desc': 'Deli Ziosa Sydney NS AUS Card xx7113 Value Date: 25/09/2017', 'classification': 'Groceries'}, {'desc': 'BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7113 Value Date: 23/09/2017', 'classification': 'Groceries'}, {'desc': 'LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 Value Date: 25/09/2017', 'classification': 'Dining Out'}, {'desc': 'Direct Debit 142619 TPG Internet DH9Q0HB22EP-03GJKN', 'classification': 'Miscellaneous'}, {'desc': 'LUO & SONG CHICKEN QPS PEAKHURST  AUS Card xx8731 Value Date: 23/09/2017', 'classification': 'Dining Out'}, {'desc': 'PLINE PH SUTHERLAND SUTHERLAND  AUS Card xx8731 Value Date: 20/09/2017', 'classification': 'Dining Ou

9

In [59]:
from io import StringIO

pd.merge(combined_df, pd.read_json(StringIO(json.dumps(classified_desc).replace("desc", "Desc"))), on='Desc', how='right')

,Date,Amount,Desc,classification
0,27/09/2017,-40.85,HARVIN SPICES HURSTVILLE NS AUS Card xx7113 Va...,Groceries
1,27/09/2017,-6.50,DELISSE SYDNEY NS AUS Card xx7113 Value Date: ...,Coffee & Snacks
2,27/09/2017,-0.80,DELISSE SYDNEY NS AUS Card xx7113 Value Date: ...,Coffee & Snacks
3,27/09/2017,-4.00,SOUTHGATE GROUP AUSTRA SYDNEY AUS Card xx7113...,Miscellaneous
4,27/09/2017,-10.00,Deli Ziosa Sydney NS AUS Card xx7113 Value Dat...,Groceries
5,27/09/2017,-430.00,BING LEE HURSTVILLE HURSTVILLE NS AUS Card xx7...,Groceries
6,27/09/2017,-4.00,LA CANTINA ON KENT SYDNEY NS AUS Card xx8731 V...,Dining Out
7,26/09/2017,-59.99,Direct Debit 142619 TPG Internet DH9Q0HB22EP-0...,Miscellaneous
8,26/09/2017,-17.85,LUO & SONG CHICKEN QPS PEAKHURST AUS Card xx8...,Dining Out
9,26/09/2017,-34.76,PLINE PH SUTHERLAND SUTHERLAND AUS Card xx873...,Dining Out
